In [81]:
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

In [82]:
dimension = 3
d_array = np.empty(dimension,dtype=np.int64)

d_array[0] = 2
d_array[1] = 2
d_array[2] = 2

L = np.prod(d_array)
print(f"We are studying a {dimension}-dimensional lattice with {L} sites")

We are studying a 3-dimensional lattice with 8 sites


In [83]:
ibc = 0
f = 3
F = np.empty(f,dtype=np.int64)
F[0] = 8
F[1] = 8
F[2] = 0
M = f*L
print(f"The dimension of the single particle Hilbert space is {M} ")

The dimension of the single particle Hilbert space is 24 


In [84]:
t = -1
site_i = []; site_j = []; hopping = []

In [85]:
def coor(index,d,n,Ns):
    den=Ns; coor=np.empty(d, dtype=int)
    for i in range(d-1, -1, -1):
        den/=n[i]
        coor[i]=index/den
        index%=den
    return coor

def coor_to_index(coor, d, d_array):
    index = 0
    m = 1
    #print(d-1)
    for dim in range(d):
        index = index + m*coor[dim]
        m = m*d_array[dim] 
    return index 

In [86]:
if(ibc == 0): #PBC
  for i in range(L): #loop over the lattice sites
    #find the d coordinates of site i
    coor_i = coor(i,dimension,d_array,L)
    #print("coordinate {}: ".format(i), coor_i)
    #find the nearest neighbors of site i
    for dim in range(dimension):
      for nn in (-1,1):
        coor_j = deepcopy( coor_i )
        coor_j[dim] = coor_i[dim] + nn
        if(coor_j[dim] < 0):
          coor_j[dim] = coor_j[dim] + d_array[dim]
        if(coor_j[dim] > d_array[dim]-1):
          coor_j[dim] = coor_j[dim] - d_array[dim]
        #print("*** nearest neighbor found in ", coor_j)
        j = coor_to_index(coor_j, dimension, d_array)
        #print("*** nn label {}:".format(j))
        site_i.append(i)
        site_j.append(j)
        hopping.append(t)
elif(ibc == 1): #OBC
  for i in range(L): #loop over the lattice sites
    coor_i = coor(i,dimension,d_array,L)
    for dim in range(dimension):
      if(coor_i[dim] == 0):
        coor_j = deepcopy( coor_i )
        coor_j[dim] = coor_i[dim] + 1
        j = coor_to_index(coor_j, dimension, d_array)
        #print("*** nearest neighbor found in ", coor_j)
        #print("*** nn label {}:".format(j))
        site_i.append(i)
        site_j.append(j)
        hopping.append(t)
      elif(coor_i[dim] == d_array[dim]-1):
        coor_j = deepcopy( coor_i )
        coor_j[dim] = coor_i[dim] - 1
        j = coor_to_index(coor_j, dimension, d_array)
        #print("*** nearest neighbor found in ", coor_j)
        #print("*** nn label {}:".format(j))
        site_i.append(i)
        site_j.append(j)
        hopping.append(t)
      else:
        for nn in (-1,1):
          coor_j = deepcopy( coor_i )
          coor_j[dim] = coor_i[dim] + nn
          j = coor_to_index(coor_j, dimension, d_array)
          #print("*** nearest neighbor found in ", coor_j)
          #print("*** nn label {}:".format(j))
          site_i.append(i)
          site_j.append(j)
          hopping.append(t)

In [87]:
Honey = np.zeros((L,L),dtype=np.complex128)
for hop in range(len(hopping)):
  Honey[site_i[hop],site_j[hop]] = hopping[hop]
eigenvalues, eigenvectors = np.linalg.eigh(Honey)
print(Honey)

[[ 0.+0.j -1.+0.j -1.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [-1.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j]
 [-1.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j -1.+0.j -1.+0.j  0.+0.j]]


In [88]:
myfile = open("1B_Eigenvalues.info","w")
for i in range(L):
  myfile.write("{} {} \n".format(i,eigenvalues[i]))
myfile.close()
# print(Honey.shape)
# print(eigenvalues.shape)
# print(eigenvalues)
# print(eigenvectors.shape[0])
print(eigenvectors)
for i in range(eigenvectors.shape[0]):
    tot = 0
    for j in range(eigenvectors.shape[0]):
        tot += eigenvectors[j][i]**2
    print(tot)

# eigenvectors.transpose()

[[ 0.35355339+0.j  0.60386886+0.j -0.08486816+0.j  0.05603391+0.j
   0.18396046+0.j  0.4000996 +0.j  0.42553362-0.j -0.35355339+0.j]
 [ 0.35355339+0.j  0.29694009+0.j  0.46774925+0.j -0.26083947+0.j
  -0.20169601+0.j -0.50981081+0.j  0.272785  +0.j  0.35355339+0.j]
 [ 0.35355339+0.j  0.1477045 +0.j -0.03002122+0.j  0.59353358+0.j
   0.47001851+0.j -0.13749171+0.j -0.36766647+0.j  0.35355339+0.j]
 [ 0.35355339+0.j -0.15922427+0.j  0.52259618+0.j  0.2766602 +0.j
  -0.45228295+0.j  0.24720292+0.j -0.33065215+0.j -0.35355339+0.j]
 [ 0.35355339+0.j  0.15922427+0.j -0.52259618+0.j -0.2766602 +0.j
  -0.45228295+0.j  0.24720292+0.j -0.33065215+0.j  0.35355339+0.j]
 [ 0.35355339+0.j -0.1477045 +0.j  0.03002122+0.j -0.59353358+0.j
   0.47001851+0.j -0.13749171+0.j -0.36766647+0.j -0.35355339+0.j]
 [ 0.35355339+0.j -0.29694009+0.j -0.46774925+0.j  0.26083947+0.j
  -0.20169601+0.j -0.50981081+0.j  0.272785  +0.j -0.35355339+0.j]
 [ 0.35355339+0.j -0.60386886+0.j  0.08486816+0.j -0.05603391+0.j
   

In [89]:
ncols = np.max(F)
Psi = np.empty((L,ncols,f), dtype = np.complex128)
for flav in range(f):
  myfile = open(f"wavefunction_flavor-{str(flav)}.info","w")
  myfile.write("# particle, site, Re(psi), Im(psi)\n")
  for part in range(F[flav]):
    for site in range(L):
      Psi[site,part,flav] = eigenvectors[site,part]
      myfile.write(f"{part+1} {site} {np.real(Psi[site,part,flav])} {np.imag(Psi[site,part,flav])}\n")
  myfile.close()

print(Psi.shape)
print(Psi)

(8, 8, 3)
[[[ 3.53553391e-001+0.00000000e+000j  3.53553391e-001+0.00000000e+000j
    2.13452835e-314+2.20331147e-314j]
  [ 6.03868857e-001+0.00000000e+000j  6.03868857e-001+0.00000000e+000j
    2.12510864e-314+2.15239063e-314j]
  [-8.48681564e-002+0.00000000e+000j -8.48681564e-002+0.00000000e+000j
    2.12517039e-314+2.12439996e-314j]
  [ 5.60339110e-002+0.00000000e+000j  5.60339110e-002+0.00000000e+000j
    2.12517039e-314+2.13811064e-314j]
  [ 1.83960461e-001+0.00000000e+000j  1.83960461e-001+0.00000000e+000j
    2.13211235e-314+2.20331045e-314j]
  [ 4.00099596e-001+0.00000000e+000j  4.00099596e-001+0.00000000e+000j
    2.12510864e-314+2.12510864e-314j]
  [ 4.25533620e-001-0.00000000e+000j  4.25533620e-001-0.00000000e+000j
    2.12503995e-314+2.20331375e-314j]
  [-3.53553391e-001+0.00000000e+000j -3.53553391e-001+0.00000000e+000j
    2.12694193e-314+2.12471114e-314j]]

 [[ 3.53553391e-001+0.00000000e+000j  3.53553391e-001+0.00000000e+000j
    2.12503995e-314+2.13452835e-314j]
  [ 2.9